In [51]:
# Husayn El Sharif
comments = """
Use XGBoost to predict whether a person will schedule an vaccine booster appointment within 7 days

Use python environment: datasci_xgb_skl_env001
"""

In [52]:
# imports

import sys
sys.path.append("..") # Add parent directory to path for imports

import os

import numpy as np
import pandas as pd

# sci-kit learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report

#scipy
from scipy.stats import pearsonr


# xgboost
from xgboost import XGBClassifier

# shap for explainability
import shap


# for making figures
import plotly.express as px
import plotly.graph_objects as go

## 📊 Column Definitions (Dataset Schema)

`data/cdc_outreach_ab_synthetic.csv`

**`person_id`**
Unique identifier for each individual in the outreach experiment.

**`age`**
Age of the individual (years).

**`sex`**
Biological sex of the individual (`M` or `F`).

**`region`**
Geographic grouping representing outreach region (e.g., `ATL-Core`, `ATL-Metro`, `North-GA`, `South-GA`).
Used to simulate location-based access and socioeconomic differences.

**`risk_score`**
Continuous score from 0–1 representing health risk or vulnerability (proxy for comorbidities and susceptibility).
Higher values indicate higher health risk.

**`barriers_index`**
Continuous index capturing access and friction to care (e.g., transportation, time constraints, technology access).
Higher values indicate **more barriers**, which generally reduce the likelihood of scheduling/completion.

**`channel`**
Outreach delivery channel used to send the message:

* `SMS`
* `Email`
* `IVR` (automated phone call)

**`weekday`**
Day of week the message was sent (0 = Monday, 6 = Sunday).

**`send_hour`**
Hour of day the message was sent (24-hour clock, typically 8–20).
Used to simulate time-of-day effects on engagement.

**`prior_cdc_interactions_90d`**
Number of prior interactions with CDC outreach in the past 90 days.
Acts as a proxy for engagement familiarity and responsiveness.

**`prior_appointments_1y`**
Number of healthcare appointments completed in the past year.
Higher values indicate stronger healthcare engagement habits.

**`missed_appointments_1y`**
Number of missed appointments in the past year.
Higher values indicate reliability or access challenges.

**`message_variant`**
Randomized A/B test assignment:

* `A` = Standard reminder (control)
* `B` = Personalized + social proof message (treatment)

**`opened`**
Binary indicator (0/1) for whether the message was opened.

**`clicked`**
Binary indicator (0/1) for whether the booking link was clicked (conditional on opening).

**`scheduled_7d`**
Binary outcome (0/1) indicating whether the individual **scheduled an appointment within 7 days** of outreach.
➡️ **Primary A/B test target variable**

**`completed_30d`**
Binary outcome (0/1) indicating whether the individual **completed the appointment within 30 days** of outreach.
➡️ Downstream real-world outcome reflecting follow-through

---

## 🧠 How this schema mirrors real-world healthcare funnels

This dataset models a realistic behavioral pipeline:

**Message sent → Opened → Clicked → Scheduled → Completed**

This allows for:

* Running causal A/B tests on **behavioral response**
* Training ML models to predict **engagement and follow-through**
* Simulating real-world public health operations and barriers to care


In [53]:
# import data
df = pd.read_csv("data/cdc_outreach_ab_synthetic.csv")
df.head()

,person_id,age,sex,region,risk_score,barriers_index,channel,weekday,send_hour,prior_cdc_interactions_90d,prior_appointments_1y,missed_appointments_1y,message_variant,opened,clicked,scheduled_7d,completed_30d
0,1,24,M,ATL-Core,0.198,1.255,Email,3,9,0,2,1,A,1,1,0,0
1,2,70,M,ATL-Metro,0.412,0.012,SMS,2,14,1,3,0,B,0,1,0,0
2,3,62,F,South-GA,0.513,0.778,SMS,6,20,1,2,2,A,0,0,0,0
3,4,47,F,ATL-Core,0.373,0.103,SMS,5,14,1,1,0,A,0,1,1,1
4,5,47,F,ATL-Core,0.327,0.942,SMS,0,20,1,0,0,B,1,0,0,0


In [54]:
# Brief EDA to understand the data
print(df.info())
print(df.describe())

print(df["scheduled_7d"].value_counts(normalize=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_id                   20000 non-null  int64  
 1   age                         20000 non-null  int64  
 2   sex                         20000 non-null  object 
 3   region                      20000 non-null  object 
 4   risk_score                  20000 non-null  float64
 5   barriers_index              20000 non-null  float64
 6   channel                     20000 non-null  object 
 7   weekday                     20000 non-null  int64  
 8   send_hour                   20000 non-null  int64  
 9   prior_cdc_interactions_90d  20000 non-null  int64  
 10  prior_appointments_1y       20000 non-null  int64  
 11  missed_appointments_1y      20000 non-null  int64  
 12  message_variant             20000 non-null  object 
 13  opened                      200

In [55]:
# Outcome (scheduled_7d) by treatment group
outcome_by_treatment = df.groupby("message_variant")["scheduled_7d"].mean()
print('Outcome (scheduled_7d) by treatment group:')
print(outcome_by_treatment)
print('\n')

# Outcome (scheduled_7d) by channel
outcome_by_channel = df.groupby("channel")["scheduled_7d"].mean().sort_values(ascending=False)
print('Outcome (scheduled_7d) by channel:')
print(outcome_by_channel)
print('\n')

# Outcome (scheduled_7d) by risk bucket
df["risk_bucket"] = pd.cut(df["risk_score"], bins=[0, 0.3, 0.6, 1.0], labels=["Low", "Medium", "High"])
risk_bucket_outcome = df.groupby("risk_bucket")["scheduled_7d"].mean()
print('Outcome (scheduled_7d) by risk bucket:')
print(risk_bucket_outcome)
print('\n')


Outcome (scheduled_7d) by treatment group:
message_variant
A    0.263048
B    0.307145
Name: scheduled_7d, dtype: float64


Outcome (scheduled_7d) by channel:
channel
SMS      0.296108
IVR      0.271169
Email    0.262051
Name: scheduled_7d, dtype: float64


Outcome (scheduled_7d) by risk bucket:
risk_bucket
Low       0.246243
Medium    0.295277
High      0.349818
Name: scheduled_7d, dtype: float64




/tmp/ipykernel_92903/819488451.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [56]:
# Correlations (numeric only)

num_cols = df.select_dtypes(include=np.number).columns

results = []

for col in num_cols:
    if col == "scheduled_7d":
        continue
    r, p = pearsonr(df[col], df["scheduled_7d"])
    results.append({
        "feature": col,
        "pearson_r": r,
        "p_value": p
    })

corr_df = (
    pd.DataFrame(results)
      .sort_values("pearson_r", ascending=False)
      .reset_index(drop=True)
)

def sig_stars(p):
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.05:
        return "*"
    else:
        return ""

corr_df["significance"] = corr_df["p_value"].apply(sig_stars)
corr_df


,feature,pearson_r,p_value,significance
0,completed_30d,0.518202,0.000000e+00,***
1,clicked,0.194056,5.967324e-169,***
2,opened,0.162198,5.843438e-118,***
3,risk_score,0.085620,7.326866e-34,***
4,age,0.071030,8.535990e-24,***
5,prior_appointments_1y,0.055325,4.892942e-15,***
6,prior_cdc_interactions_90d,0.035214,6.316301e-07,***
7,person_id,0.011096,1.165985e-01,
8,send_hour,0.002997,6.717376e-01,
9,weekday,-0.000380,9.571547e-01,


## Goal: Pre-Send Targeting

XGBoost will be used to answer the question:

“Who should CDC target (with message variant B)?”

**This is a counterfactual question:**
We're asking who would be more likely to schedule an appointment (within 7 days of receiving) if they receive the personalized message (B) vs standard (A).

The XGBoost model needs to learn:

* Baseline propensity to schedule
* How treatment (message_variant) changes that probability
* How that effect varies by person (heterogeneity)

P(scheduled_7d = 1 | X, message_variant)
Where X = demographics, risk, barriers, history, etc.

For the same person:
  Predict outcome if message_variant = A
  Predict outcome if message_variant = B
  Take the difference → uplift

This allows us to simulate:
**“If I send B instead of A to this person, how much does their probability of scheduling increase?”**


In [57]:
# -------------------------
# 1) Define target + features (PRE-SEND)
# -------------------------
target = "scheduled_7d"

features_pre_send = [
    "age", "sex", "region", "risk_score", "barriers_index", "channel",
    "weekday", "send_hour",
    "prior_cdc_interactions_90d", "prior_appointments_1y", "missed_appointments_1y",
    "message_variant"  # include so we can simulate A vs B later
]

X = df[features_pre_send].copy()
y = df[target].copy()

# Categorical vs numeric columns
cat_features = ["sex", "region", "channel", "message_variant"]
num_features = [c for c in features_pre_send if c not in cat_features]

# -------------------------
# 2) Train/test split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y, # ensure same class balance in train/test splits (important for imbalanced data)
)

# -------------------------
# 3) Preprocess: One-hot encode categoricals, pass through numerics
# -------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="first"), cat_features), # drop="first" to avoid mutually exclusive dummy variable trap
        ("num", "passthrough", num_features)
    ]
)

# -------------------------
# 4) XGBoost model
# -------------------------
"""xgb_model = XGBClassifier(
    n_estimators=400, # more trees for better performance (default is 100)
    max_depth=5, # deeper trees can capture more complex patterns (default is 3)
    learning_rate=0.05, # lower learning rate for better performance (default is 0.1)
    subsample=0.8, # subsample rows for each tree to prevent overfitting
    colsample_bytree=0.8, # subsample columns for each tree to prevent overfitting
    reg_lambda=1.0, # L2 regularization to prevent overfitting
    eval_metric="logloss", # use logloss for binary classification
    random_state=42
)"""

# include class weighting to handle class imbalance 
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

xgb_model = XGBClassifier(
    n_estimators=400,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=pos_weight,
    eval_metric="auc",
    random_state=42
)



In [58]:
# Full pipeline
pipe = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("model", xgb_model)
    ]
)


In [59]:
# -------------------------
# 5) Fit + evaluate
# -------------------------
pipe.fit(X_train, y_train)

y_pred_proba = pipe.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))

ROC-AUC: 0.6621285241074715
              precision    recall  f1-score   support

           0       0.79      0.67      0.72      3575
           1       0.40      0.56      0.47      1425

    accuracy                           0.64      5000
   macro avg       0.60      0.61      0.60      5000
weighted avg       0.68      0.64      0.65      5000



In [ ]:
# tune the decision threshold for better recall (catch more positives)
# optimizing for message targeting, not accuracy. Lower the threshold to improve recall:

threshold = 0.35  # try 0.3–0.45
y_pred_tuned = (y_pred_proba >= threshold).astype(int)

print(classification_report(y_test, y_pred_tuned))

comment= """
The pre-send XGBoost model was optimized for outreach coverage rather than accuracy. 
With class weighting and a tuned decision threshold (0.35), the model achieves 83 percent recall 
for likely schedulers, capturing the majority of potential converters at the expense of precision. 
This operating point reflects real-world public health outreach objectives, where missing high-propensity 
individuals is more costly than contacting some low-propensity individuals. 
Ranking performance (ROC-AUC ≈ 0.66) remains stable, indicating that threshold tuning changes 
actionability without degrading the model’s ability to prioritize individuals.

"""

              precision    recall  f1-score   support

           0       0.84      0.36      0.50      3575
           1       0.34      0.83      0.48      1425

    accuracy                           0.49      5000
   macro avg       0.59      0.60      0.49      5000
weighted avg       0.70      0.49      0.50      5000



## 1️⃣ SHAP Explanations (Why the model predicts scheduling)

Goal: show **global drivers** (which features matter most) and **local explanations** (why a specific person is predicted high/low).

In [61]:
# Sample for speed
SHAP_SAMPLE = 2000
X_shap = X_test.sample(min(SHAP_SAMPLE, len(X_test)), random_state=42)

# Transform with fitted preprocessor
X_shap_trans = pipe.named_steps["prep"].transform(X_shap)
feature_names = pipe.named_steps["prep"].get_feature_names_out()

explainer = shap.TreeExplainer(pipe.named_steps["model"])
shap_values = explainer.shap_values(X_shap_trans)  # shape: (n_samples, n_features)

In [65]:
# Mean absolute SHAP per feature
mean_abs_shap = np.abs(shap_values).mean(axis=0)

shap_importance_df = (
    pd.DataFrame({
        "feature": feature_names,
        "mean_abs_shap": mean_abs_shap
    })
    .sort_values("mean_abs_shap", ascending=False)
    .head(20)
)

fig = px.bar(
    shap_importance_df,
    x="mean_abs_shap",
    y="feature",
    orientation="h",
    title="Global Feature Importance (SHAP – XGBoost, Pre-Send Model)"
)
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

In [66]:
# Compute mean signed SHAP per feature
mean_signed_shap = shap_values.mean(axis=0)

dir_df = (
    pd.DataFrame({
        "feature": feature_names,
        "mean_signed_shap": mean_signed_shap,
        "mean_abs_shap": np.abs(shap_values).mean(axis=0)
    })
    .sort_values("mean_abs_shap", ascending=False)
    .head(15)
)

fig = px.bar(
    dir_df,
    x="mean_signed_shap",
    y="feature",
    orientation="h",
    title="Directional Feature Influence (Mean SHAP, Pre-Send XGBoost)"
)

fig.add_vline(x=0, line_dash="dot")
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

### Directional Feature Influence (Mean SHAP, Pre-Send XGBoost)

**Caption**
Mean signed SHAP values for the pre-send XGBoost model predicting appointment scheduling within 7 days. Positive values indicate features that, on average, increase the predicted probability of scheduling, while negative values indicate features that decrease it. The dashed vertical line at zero represents no net directional effect.

**Interpretation**
Access barriers (`barriers_index`) are the dominant negative driver of predicted scheduling probability, indicating that structural and logistical constraints strongly suppress follow-through regardless of messaging. In contrast, higher health risk (`risk_score`) and assignment to the personalized message (`message_variant_B`) contribute positively to predicted scheduling, though their average effects are smaller than the impact of access barriers. Prior missed appointments and older age also trend negatively, reflecting persistent behavioral and access challenges. Overall, the model suggests that while personalized outreach helps, reducing access barriers is likely to yield larger gains in scheduling than messaging alone.

---

### Key Finding

> SHAP directionality shows that access barriers overwhelmingly suppress scheduling probability, while higher risk and personalized messaging (message variant B) modestly increase predicted follow-through. This highlights that **structural access constraints dominate messaging effects**.


## 🎯 Actionable Insights

Based on the A/B testing results and SHAP-driven model interpretability, the following operational actions are recommended to improve public health outreach effectiveness:

**1️⃣ Pair personalized messaging with barrier reduction**
While the personalized message (Variant B) increases scheduling probability, SHAP analysis shows that **access barriers (`barriers_index`) are the dominant negative driver** of follow-through. Messaging alone is unlikely to fully overcome logistical constraints.
**Operational action:** Combine Message B with barrier-reduction interventions (e.g., transportation vouchers, extended clinic hours, simplified booking links) for high-barrier individuals.

**2️⃣ Prioritize high-risk populations for personalized outreach**
Higher `risk_score` is associated with increased responsiveness to outreach.
**Operational action:** Allocate personalized messaging (Variant B) preferentially to higher-risk cohorts to maximize both public health impact and conversion efficiency.

**3️⃣ Target individuals with poor appointment adherence using multi-touch strategies**
Individuals with more `missed_appointments_1y` show lower predicted scheduling probability.
**Operational action:** For historically unreliable groups, deploy multi-touch outreach (e.g., SMS + IVR follow-up) or human-assisted scheduling to improve completion rates.

**4️⃣ Use uplift targeting to allocate limited resources**
Counterfactual predictions (`p_B − p_A`) identify individuals who benefit most from personalized messaging.
**Operational action:** When budgets or capacity are constrained, target Message B to the top uplift decile/quantile to maximize conversions per outreach dollar.



---

### Summary

> The analysis suggests that personalized messaging should be deployed selectively and paired with barrier-reduction strategies, as structural access constraints dominate messaging effects in determining follow-through.



In [67]:
# Identify index of risk_score feature
feat = "num__risk_score"
j = list(feature_names).index(feat)

risk_vals = X_shap_trans[:, j].toarray().ravel() if hasattr(X_shap_trans[:, j], "toarray") else X_shap_trans[:, j]
risk_shap = shap_values[:, j]

df_risk = pd.DataFrame({
    "risk_score": risk_vals,
    "shap_value": risk_shap
})

fig = px.scatter(
    df_risk,
    x="risk_score",
    y="shap_value",
    trendline="lowess",
    title="SHAP vs Risk Score (Pre-Send XGBoost)"
)
fig.add_hline(y=0, line_dash="dot")
fig.show()

## SHAP vs. Risk Score plot

Higher `risk_score` **increases** the predicted probability of scheduling; however, the effect is **nonlinear and heterogeneous**.

### Interpretation of the pattern you’re seeing

* At **low risk scores (~0–0.3)**
  → SHAP values are mostly **negative**
  → Low-risk individuals are less likely to schedule (lower urgency)

* At **moderate risk (~0.4–0.6)**
  → SHAP values cluster around zero
  → Neutral effect on scheduling

* At **high risk (~0.6–1.0)**
  → SHAP values trend **positive**
  → Higher-risk individuals are more likely to schedule

So the relationship is:

> **Risk score increases scheduling probability, but only after a threshold — and interacts with other factors (e.g., barriers, channel).**

Because many people are low-risk, the **mean signed SHAP** gets pulled negative overall, even though the **marginal effect of increasing risk is positive at the high end**.

---

**Caption**
SHAP dependence plot for `risk_score` in the pre-send XGBoost model. Positive SHAP values indicate higher predicted probability of scheduling within 7 days. The relationship is nonlinear: low risk scores are associated with reduced predicted scheduling, while higher risk scores increase predicted scheduling probability, reflecting heterogeneous treatment effects across the population.

---

## 🧠  Interpretation

> SHAP dependence analysis shows a nonlinear relationship between health risk and scheduling behavior. Low-risk individuals are less likely to schedule in response to outreach, while higher-risk individuals exhibit increasing predicted scheduling probability. This indicates that personalized outreach is more effective among higher-risk populations and supports prioritizing high-risk individuals for targeted interventions.




In [72]:
# Use XGBoost results for targeting

# Copy a cohort of people BEFORE sending
X_people = df[features_pre_send].copy()

# Scenario A: everyone gets A
X_people_A = X_people.copy()
X_people_A["message_variant"] = "A"

# Scenario B: everyone gets B
X_people_B = X_people.copy()
X_people_B["message_variant"] = "B"

# Predict probabilities
p_A = pipe.predict_proba(X_people_A)[:, 1]
p_B = pipe.predict_proba(X_people_B)[:, 1]

uplift = p_B - p_A

targeting_df = X_people.copy()
targeting_df["p_A"] = p_A
targeting_df["p_B"] = p_B
targeting_df["uplift"] = uplift

targeting_df.sort_values("uplift", ascending=False).head(100)

,age,sex,region,risk_score,barriers_index,channel,weekday,send_hour,prior_cdc_interactions_90d,prior_appointments_1y,missed_appointments_1y,message_variant,p_A,p_B,uplift
15529,61,F,South-GA,0.133,1.493,IVR,6,19,1,1,0,B,0.439226,0.718338,0.279112
9051,55,F,ATL-Core,0.099,-0.867,SMS,3,10,0,0,0,B,0.372162,0.601581,0.229419
13701,85,F,ATL-Core,0.688,0.430,SMS,3,10,0,1,0,B,0.450477,0.669941,0.219464
3008,60,M,ATL-Core,0.219,1.502,IVR,6,20,3,1,0,B,0.292974,0.501772,0.208798
6839,75,M,ATL-Core,0.572,1.467,IVR,5,11,2,4,2,B,0.267333,0.474779,0.207446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10685,77,F,ATL-Core,0.511,0.822,SMS,1,8,4,0,0,B,0.447355,0.602404,0.155049
8356,81,F,ATL-Core,0.343,0.397,SMS,3,8,1,1,1,A,0.362787,0.517684,0.154898
10593,85,M,North-GA,0.713,0.040,SMS,1,8,4,1,0,B,0.671531,0.826176,0.154645
5184,71,M,South-GA,0.404,-0.932,SMS,1,10,2,1,1,B,0.517000,0.671592,0.154592
